In [119]:
#import packages
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

In [120]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24

In [121]:
# Read excel file which contains Non renewable generators data
bus=pd.read_excel('data_exercise_2.xlsx',sheet_name='buses',header=0)

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



In [122]:
#read bus file to verify the code
bus

,bus,v_nom,carrier,x,y
0,north_de,380,AC,53.558119,9.598188
1,east_de,380,AC,52.205416,9.754593
2,west_de,380,AC,52.598466,9.101421


In [123]:
#add buses to the network

network.madd(
    "Bus",
    names= bus.bus,
    bus=list(bus.v_nom),
    carrier=list(bus.carrier),
    x=list(bus.x),
    y=list(bus.y),
)
network.buses

C:\Users\suhas\AppData\Local\Temp\ipykernel_10684\1741324144.py:3: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.

Index(['north_de', 'east_de', 'west_de'], dtype='object', name='bus')


,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network,bus
Bus,,,,,,,,,,,,,
north_de,1.0,,53.558119,9.598188,AC,,1.0,0.0,inf,PQ,,,380
east_de,1.0,,52.205416,9.754593,AC,,1.0,0.0,inf,PQ,,,380
west_de,1.0,,52.598466,9.101421,AC,,1.0,0.0,inf,PQ,,,380


In [124]:
# Read excel file which contains Non generators data
generators=pd.read_excel('data_exercise_2.xlsx',sheet_name='generators')

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



In [125]:
#read the generators file
generators

,generator,p_nom,p_nom_max,bus,p_nom_extendable,marginal_cost
0,gen_no1,31,31,north_de,False,10
1,gen_no2,19,19,east_de,False,15
2,gen_no3,10,10,west_de,False,25


In [126]:
#add generators to the network

network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),  # GW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

C:\Users\suhas\AppData\Local\Temp\ipykernel_10684\1756491760.py:3: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,e_sum_min,e_sum_max,q_set,sign,carrier,marginal_cost,marginal_cost_quadratic,active,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,stand_by_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gen_no1,north_de,PQ,,31.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no1,10.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no2,east_de,PQ,,19.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no2,15.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no3,west_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no3,25.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [127]:
#read generatores in network
generators

,generator,p_nom,p_nom_max,bus,p_nom_extendable,marginal_cost
0,gen_no1,31,31,north_de,False,10
1,gen_no2,19,19,east_de,False,15
2,gen_no3,10,10,west_de,False,25


In [128]:
#add generators to the network

network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),  # GW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

C:\Users\suhas\AppData\Local\Temp\ipykernel_10684\1756491760.py:3: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,e_sum_min,e_sum_max,q_set,sign,carrier,marginal_cost,marginal_cost_quadratic,active,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,stand_by_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gen_no1,north_de,PQ,,31.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no1,10.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no2,east_de,PQ,,19.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no2,15.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no3,west_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no3,25.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [129]:
#read generatores in network
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,e_sum_min,e_sum_max,q_set,sign,carrier,marginal_cost,marginal_cost_quadratic,active,build_year,lifetime,capital_cost,efficiency,committable,start_up_cost,shut_down_cost,stand_by_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
gen_no1,north_de,PQ,,31.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no1,10.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no2,east_de,PQ,,19.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no2,15.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no3,west_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,0.0,-inf,inf,0.0,1.0,gen_no3,25.0,0.0,True,0,inf,0.0,1.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [130]:
# add load
# Read excel file which contains Non renewable generators data
load=pd.read_excel('data_exercise.xlsx',sheet_name='load',header=0)

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



In [131]:
#read load
load

,bus,p_set,carrier
0,north_de,10,AC
1,east_de,10,AC
2,west_de,40,AC


In [132]:
#add laod to the network

network.madd(
    "Load",
    names= load.bus,
    bus=list(load.bus),
    p_set=list(load.p_set),  # GW
) 
network.loads


C:\Users\suhas\AppData\Local\Temp\ipykernel_10684\1225564043.py:3: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
north_de,north_de,,,10.0,0.0,-1.0,True
east_de,east_de,,,10.0,0.0,-1.0,True
west_de,west_de,,,40.0,0.0,-1.0,True


In [133]:
#add transmission lines
# Read excel file which contains Non renewable generators data
lines=pd.read_excel('data_exercise_2.xlsx',sheet_name='Trans lines',header=0)

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

c:\Users\suhas\anaconda3\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



In [134]:
lines.head()

,bus0,bus1,s_nom,s_nom_extendable,length
0,north_de,east_de,14,False,400
1,east_de,west_de,14,False,400
2,west_de,north_de,14,False,400


In [135]:
# Add trans line connection between the buses
for index, row in lines.iterrows():
    network.add(
        "Line",
        "{}_{}".format(row['bus0'],row['bus1']),
        bus0=row['bus0'],
        bus1=row['bus1'],
        s_nom=row['s_nom'],
        x=0.0001,
    )

In [136]:
network.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,active,build_year,lifetime,length,carrier,terrain_factor,num_parallel,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
north_de_east_de,north_de,east_de,,0.0001,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
east_de_west_de,east_de,west_de,,0.0001,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
west_de_north_de,west_de,north_de,,0.0001,0.0,0.0,0.0,14.0,0.0,False,0.0,inf,1.0,0.0,True,0,inf,0.0,,1.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
# Run the power flow calculation
network.optimize(solver_name='highs')

Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['gen_no1', 'gen_no2', 'gen_no3'], dtype='object', name='Generator')
Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['gen_no1', 'gen_no2', 'gen_no3'], dtype='object', name='Generator')
c:\Users\suhas\anaconda3\Lib\site-packages\linopy\common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the

('warning', 'infeasible')

In [138]:
network.generators_t.p

Generator
snapshot
0


In [139]:
network.lines_t.p0

Line
snapshot
0


In [142]:
network.pf()

{'n_iter': SubNetwork  0
 snapshot     
 0           2,
 'error': SubNetwork             0
 snapshot                
 0           2.083399e-07,
 'converged': SubNetwork     0
 snapshot        
 0           True}

In [143]:
for line in network.lines.index:
    loading = abs(network.lines_t.p0.loc[:, line])  # Power flow in each line
    capacity = network.lines.at[line, "s_nom"]
    if loading.max() > capacity:
        print(f"Line {line} is overloaded. Max Loading: {loading.max()} GW, Capacity: {capacity} GW")
    else:
        print(f"Line {line} is within capacity.Max Loading: {loading.max()} GW, Capacity: {capacity} GW")

Line north_de_east_de is overloaded. Max Loading: 20.00000444426586 GW, Capacity: 14.0 GW
Line east_de_west_de is within capacity.Max Loading: 10.00000444407788 GW, Capacity: 14.0 GW
Line west_de_north_de is overloaded. Max Loading: 29.999995554762815 GW, Capacity: 14.0 GW
